In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import tensorflow as tf
import pickle
import sys
import yaml

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, TimeDistributed, Bidirectional
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.utils import plot_model
from keras import optimizers
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from tensorflow.compat.v1.keras import backend as K
from keras import backend as K # add

import datetime
import math
import hashlib
import time
import os
from datagenerator import DataGenerator
import random
import matplotlib.pyplot as plt

#Disable randomization
seed_value=2020 #10 #

# add
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"  # specify which GPU(s) to be used

#Disable randomisation
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

session_conf.gpu_options.allow_growth = True # add

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [2]:
def loadConfig():
    with open('./light_config.yaml', "r") as ymlfile: #GE, hulk, slowhttp, slowloris, heartbleed
        cfg = yaml.load(ymlfile)
    return cfg

config = loadConfig()

/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
df_normal = pd.read_csv(config['metadata']['uniqueID'] + '/' + config['metadata']['artefact'] + '/' + 'N1.csv')
print(len(df_normal))
df_normal.head()

3262


,Source IP,Dest IP,Protocols,Info,Input,Histo
0,126.120.0.39,71.126.222.64,raw:ip:tcp,"80 → 46689 [SYN, ACK] Seq=0 Ack=1 Win=65535 Le...",367<SEP>295<SEP>367<SEP>549<SEP>367<SEP>101<SE...,"80 → 46689 [SYN, ACK] Seq=0 Ack=1 Win=65535 Le..."
1,126.120.0.39,71.126.222.64,raw:ip:tcp,"80 → 59856 [PSH, ACK] Seq=17145 Ack=330 Win=65...",162<SEP>289<SEP>17<SEP>17<SEP>162<SEP>289<SEP>...,"80 → 59856 [PSH, ACK] Seq=17145 Ack=330 Win=65..."
2,126.120.0.39,71.126.222.64,raw:ip:tcp,80 → 45234 [ACK] Seq=1449 Ack=413 Win=65123 Le...,545<SEP>383<SEP>91<SEP>256<SEP>338<SEP>542<SEP...,80 → 45234 [ACK] Seq=1449 Ack=413 Win=65123 Le...
3,126.120.0.39,71.126.222.64,raw:ip:tcp,80 → 35187 [ACK] Seq=692 Ack=646 Win=64891 Len...,233<SEP>368<SEP>256<SEP>367<SEP>671<SEP>549<SE...,80 → 35187 [ACK] Seq=692 Ack=646 Win=64891 Len...
4,126.120.0.39,71.126.222.64,raw:ip:tcp,80 → 35194 [ACK] Seq=1 Ack=649 Win=64887 Len=0...,58<SEP>358<SEP>693<SEP>358<SEP>693<SEP>87<SEP>...,80 → 35194 [ACK] Seq=1 Ack=649 Win=64887 Len=0...


In [5]:
print("Unique Source IP: ", len(df_normal['Source IP'].unique()))
print("Unique Dest IP: ", len(df_normal['Dest IP'].unique()))

df_normal['SourceDest'] = df_normal["Source IP"] + df_normal["Dest IP"]
print("Unique SourceDest IP: ", len(df_normal['SourceDest'].unique()))

Unique Source IP:  125
Unique Dest IP:  190
Unique SourceDest IP:  313


In [6]:
# Entire test set. 50622 rows
df_test = pd.read_csv(config['metadata']['uniqueID'] + '/' + config['metadata']['artefact'] + '/' + 'TEST_full.csv')
len(df_test)

50622

In [7]:
# Creating small test attack set 
num_attack = int(len(df_normal)) 
df_test = df_test[:num_attack]
# dftest_small.to_csv(config['metadata']['uniqueID'] + '/' + 
#                     config['metadata']['artefact'] + '/' + 'TEST.csv', index=False)
# len(dftest_small) 
# dftest_small.head()

# df_test = pd.read_csv(config['metadata']['uniqueID'] + '/' + config['metadata']['artefact'] + '/' + 'TEST.csv')
print(len(df_test))
df_test.head()

3262


,Source IP,Dest IP,Protocols,Info,Input,Histo
0,115.110.73.128,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xa9ee, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0xa9ee, seq=1280/5, tt..."
1,115.110.73.128,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xa9ee, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0xa9ee, seq=1280/5, tt..."
2,115.110.73.128,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xa9ee, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0xa9ee, seq=1280/5, tt..."
3,115.124.203.55,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt..."
4,115.124.203.55,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt..."


In [8]:
print("Unique Source IP: ", len(df_test['Source IP'].unique()))
print("Unique Dest IP: ", len(df_test['Dest IP'].unique()))

df_test['SourceDest'] = df_test["Source IP"] + df_test["Dest IP"]
print("Unique SourceDest IP: ", len(df_test['SourceDest'].unique()))
# df_test.head()

Unique Source IP:  422
Unique Dest IP:  1
Unique SourceDest IP:  422


In [9]:
# add labels for evaluation
df_test['Attack'] = 1
df_normal['Attack'] = 0

In [10]:
# join and shuffle test attack and normal data
df_test = df_test.append(df_normal).sample(frac=1, replace=False, random_state=39).reset_index(drop=True)
print("size of df_test: ", len(df_test))
df_test.head()

size of df_test:  6524


,Source IP,Dest IP,Protocols,Info,Input,Histo,SourceDest,Attack
0,51.173.229.255,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt...",51.173.229.25571.126.222.64,0
1,51.173.229.255,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt...",51.173.229.25571.126.222.64,0
2,192.95.27.190,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xc6f8, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0xc6f8, seq=1280/5, tt...",192.95.27.19071.126.222.64,0
3,192.120.148.227,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt...",192.120.148.22771.126.222.64,0
4,132.25.219.236,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt...",132.25.219.23671.126.222.64,1


In [11]:
# Small test set. 50:50
df_test.to_csv(config['metadata']['uniqueID'] + '/' + 
                    config['metadata']['artefact'] + '/' + 'TEST.csv', index=False)
# df_test = pd.read_csv(config['metadata']['uniqueID'] + '/' + config['metadata']['artefact'] + '/' + 'TEST_mix50.csv')
print(len(df_test))
df_test.head()

6524


,Source IP,Dest IP,Protocols,Info,Input,Histo,SourceDest,Attack
0,51.173.229.255,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt...",51.173.229.25571.126.222.64,0
1,51.173.229.255,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt...",51.173.229.25571.126.222.64,0
2,192.95.27.190,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xc6f8, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0xc6f8, seq=1280/5, tt...",192.95.27.19071.126.222.64,0
3,192.120.148.227,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt...",192.120.148.22771.126.222.64,0
4,132.25.219.236,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",60<JOIN>0x00000000<JOIN>0.0<JOIN>0.0<JOIN>0<JO...,"Echo (ping) request id=0x0200, seq=1280/5, tt...",132.25.219.23671.126.222.64,1
